In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB7
from google.colab.patches import cv2_imshow
import os
import cv2
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
zip_file_path = '/content/drive/MyDrive/dataset.zip'
destination_path = '/content'
!unzip -q /content/drive/MyDrive/dataset.zip -d /content

In [ ]:
def get_dataset(ds_type):
  images = []
  labels = []
  for (dirpath, dirname, filenames) in os.walk(f"/content/{ds_type}"):
    for file_name in filenames:
      img_name = file_name.split(".")
      name = img_name[0]
      extension = img_name[1]
      if(extension == "jpg"):
        label_name = dirpath+"/"+name+".txt"
        img_file_name = dirpath+"/"+name+".jpg"
        f = open(label_name, "r")
        line1 = f.readline()
        line2 = f.readline()
        cid1, x1, y1, w1, h1 = list(map(float, line1.split()))
        cid2, x2, y2, w2, h2 = list(map(float, line2.split()))
        im = cv2.imread(img_file_name)
        im = cv2.resize(im, (182, 290), interpolation = cv2.INTER_LINEAR)
        images.append(im)
        labels.append([cid1, x1, y1, w1, h1, cid2, x2, y2, w2, h2])
  return np.array(images), np.array(labels).reshape(-1, num_classes)

In [ ]:
import tensorflow as tf

def iou_loss(y_true, y_pred):
  box1 = (y_pred[1]*290,y_pred[2]*182,y_pred[3]*290,y_pred[4]*182)
  box2 = (y_true[1]*290,y_true[2]*182,y_true[3]*290,y_true[4]*182)
  box3 = (y_pred[1]*290,y_pred[2]*182,y_pred[3]*290,y_pred[4]*182)
  box4 = (y_true[1],y_true[2],y_true[3],y_true[4])
  # Get coordinates of the intersection rectangle
  x1 = max(box1[0], box2[0])
  y1 = max(box1[1], box2[1])
  x2 = min(box1[0] + box1[2], box2[0] + box2[2])
  y2 = min(box1[1] + box1[3], box2[1] + box2[3])
  # Calculate area of the intersection rectangle
  intersection_area = max(0, x2 - x1) * max(0, y2 - y1)
  # Calculate area of the union rectangle
  box1_area = box1[2] * box1[3]
  box2_area = box2[2] * box2[3]
  union_area = box1_area + box2_area - intersection_area
  # Calculate IoU
  iou1 = intersection_area / union_area
  x1 = max(box3[0], box4[0])
  y1 = max(box3[1], box4[1])
  x2 = min(box3[0] + box3[2], box4[0] + box4[2])
  y2 = min(box3[1] + box3[3], box4[1] + box4[3])
  # Calculate area of the intersection rectangle
  intersection_area = max(0, x2 - x1) * max(0, y2 - y1)
  # Calculate area of the union rectangle
  box3_area = box3[2] * box3[3]
  box4_area = box4[2] * box4[3]
  union_area = box3_area + box4_area - intersection_area
  # Calculate IoU
  iou2 = intersection_area / union_area
  return iou1+iou2

In [ ]:
# Define input shape and number of classes
input_shape = (290, 182, 3)
num_classes = 10

# Load pre-trained EfficientNetB7 model with ImageNet weights
base_model = EfficientNetB7(include_top=False, weights='imagenet', input_shape=input_shape)

# Freeze base model layers
for layer in base_model.layers[:len(base_model.layers)-5]:
    layer.trainable = False

# Add custom output layers for OCR
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(num_classes, activation='relu')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=x)
history = model.summary(expand_nested=True)
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
im = cv2.imread("/content/6.jpg")
im = cv2.resize(im, (182, 290), interpolation = cv2.INTER_LINEAR)
im = np.expand_dims(im, axis=0)  # Add batch dimension
output = model.predict(im)
cv2_imshow(output)